<a href="https://colab.research.google.com/github/Mini-project4ppl/Mini_Project/blob/master/DN_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models


# Set the path to your dataset
data_dir = "/content/drive/MyDrive/DR_CAUG"

# Define the transformations applied to the input images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
dataset = ImageFolder(data_dir, transform=transform)

# Define the DataLoader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Load the pre-trained DenseNet201 model
model = models.densenet201(pretrained=True)

# Replace the classifier to match the number of classes in your dataset
num_classes = len(dataset.classes)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check if a GPU is available and move the model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Save the trained model to your Google Drive
drive_dir = "/content/drive/MyDrive/trained_models"
model_name = "trained_model1.pth"
model_path = f"{drive_dir}/{model_name}"
torch.save(model.state_dict(), model_path)


In [ ]:

import os
import csv
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Create an instance of the DenseNet201 model
model = models.densenet201()

# Load the pretrained weights of your model
pretrained_weights = torch.load("/content/drive/MyDrive/trained_models/trained_model2.pth")

# Remove the 'module.' prefix from the state_dict keys (if present)
if list(pretrained_weights.keys())[0].startswith('module.'):
    pretrained_weights = {k[7:]: v for k, v in pretrained_weights.items()}

# Remove the last layer of the current model classifier
model.classifier = torch.nn.Identity()

# Load the pretrained weights into the model (excluding the last layer)
model.load_state_dict(pretrained_weights, strict=False)

# Add a new classifier layer with the correct number of output classes
num_classes = 5
model.classifier = torch.nn.Linear(1920, num_classes)

# Set the model to evaluation mode
model.eval()

# Preprocess the input image
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Set the path to the folder containing the images
image_folder_path = "/content/drive/MyDrive/resized_train"

# Set the path to the output CSV file
csv_file_path = "/content/drive/MyDrive/Results/Results2_model2.csv"

# Create a CSV file and write the header row
with open(csv_file_path, mode='w') as csv_file:
    fieldnames = ['image_name', 'predicted_severity']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    # Loop through all the image files in the folder
    for filename in os.listdir(image_folder_path):
        # Load and preprocess the input image
        input_image = Image.open(os.path.join(image_folder_path, filename)).convert("RGB")
        #print(f"Input image shape: {input_image.size}")
        input_tensor = transform(input_image)
        #print(f"Input tensor shape: {input_tensor.shape}")
        input_batch = input_tensor.unsqueeze(0)

        # Check if a GPU is available and move the model and input to GPU if possible
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        input_batch = input_batch.to(device)

        # Perform the forward pass
        with torch.no_grad():
            output = model(input_batch)

        # Get the predicted severity level
        _, predicted_index = torch.max(output, 1)

        severity_levels = ["No DR", "Mild", "Moderate", "Severe", "Proliferative DR"]
        if predicted_index.item() < len(severity_levels):
            predicted_severity = severity_levels[predicted_index.item()]
        else:
            predicted_severity = "Unknown"

        print(f"Image: {filename}, Predicted Severity Level: {predicted_severity}")

        # Write the image name and predicted severity to the CSV file
        writer.writerow({'image_name': filename, 'predicted_severity': predicted_severity})


Streaming output truncated to the last 5000 lines.
Image: 14842_right.jpeg, Predicted Severity Level: Proliferative DR
Image: 14852_right.jpeg, Predicted Severity Level: Proliferative DR
Image: 14849_right.jpeg, Predicted Severity Level: Proliferative DR
Image: 14844_left.jpeg, Predicted Severity Level: Mild
Image: 14855_left.jpeg, Predicted Severity Level: Mild
Image: 14842_left.jpeg, Predicted Severity Level: Mild
Image: 14876_right.jpeg, Predicted Severity Level: Mild
Image: 14867_left.jpeg, Predicted Severity Level: Mild
Image: 14858_left.jpeg, Predicted Severity Level: Severe
Image: 14843_right.jpeg, Predicted Severity Level: Proliferative DR
Image: 14846_right.jpeg, Predicted Severity Level: Severe
Image: 14850_right.jpeg, Predicted Severity Level: Proliferative DR
Image: 14852_left.jpeg, Predicted Severity Level: Proliferative DR
Image: 14858_right.jpeg, Predicted Severity Level: Severe
Image: 14849_left.jpeg, Predicted Severity Level: Proliferative DR
Image: 14844_right.jpeg, P

In [ ]:
import os
import csv
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Load and configure the first model
model_1 = models.densenet201()
pretrained_weights_1 = torch.load("/content/drive/MyDrive/trained_models/trained_model1.pth")
model_1.classifier = torch.nn.Identity()
model_1.load_state_dict(pretrained_weights_1, strict=False)
num_classes = 5
model_1.classifier = torch.nn.Linear(1920, num_classes)
model_1.eval()

# Load and configure the second model
model_2 = models.densenet201()
pretrained_weights_2 = torch.load("/content/drive/MyDrive/trained_models/trained_model2.pth")
model_2.classifier = torch.nn.Identity()
model_2.load_state_dict(pretrained_weights_2, strict=False)
model_2.classifier = torch.nn.Linear(1920, num_classes)
model_2.eval()

# Load and configure the third model
model_3 = models.densenet201()
pretrained_weights_3 = torch.load("/content/drive/MyDrive/trained_models/trained_model3.pth")
model_3.classifier = torch.nn.Identity()
model_3.load_state_dict(pretrained_weights_3, strict=False)
model_3.classifier = torch.nn.Linear(1920, num_classes)
model_3.eval()

# Preprocess the input image
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Set the path to the folder containing the images
image_folder_path = "/content/drive/MyDrive/resized_train"

# Set the path to the output CSV file
csv_file_path = "/content/drive/MyDrive/Results/Results_combined_models.csv"

# Create a CSV file and write the header row
with open(csv_file_path, mode='w') as csv_file:
    fieldnames = ['image_name', 'predicted_severity']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    # Loop through all the image files in the folder
    for filename in os.listdir(image_folder_path):
        # Load and preprocess the input image
        input_image = Image.open(os.path.join(image_folder_path, filename)).convert("RGB")
        input_tensor = transform(input_image)
        input_batch = input_tensor.unsqueeze(0)

        # Check if a GPU is available and move the models and input to GPU if possible
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model_1 = model_1.to(device)
        model_2 = model_2.to(device)
        model_3 = model_3.to(device)
        input_batch = input_batch.to(device)

        # Perform the forward pass for each model
        with torch.no_grad():
            output_1 = model_1(input_batch)
            output_2 = model_2(input_batch)
            output_3 = model_3(input_batch)

        # Combine the predictions of all models (averaging)
        combined_output = (output_1 + output_2 + output_3) / 3.0

        # Get the predicted severity level
        _, predicted_index = torch.max(combined_output, 1)

        severity_levels = ["No DR", "Mild", "Moderate", "Severe", "Proliferative DR"]
        if predicted_index.item() < len(severity_levels):
            predicted_severity = severity_levels[predicted_index.item()]
        else:
            predicted_severity = "Unknown"

        print(f"Image: {filename}, Predicted Severity Level: {predicted_severity}")

        # Write the image name and predicted severity to the CSV file
        writer.writerow({'image_name': filename, 'predicted_severity': predicted_severity})


Image: 8885_left.jpeg, Predicted Severity Level: No DR
Image: 8883_right.jpeg, Predicted Severity Level: Severe
Image: 8882_right.jpeg, Predicted Severity Level: Mild
Image: 8889_left.jpeg, Predicted Severity Level: Mild
Image: 8885_right.jpeg, Predicted Severity Level: Mild
Image: 8893_left.jpeg, Predicted Severity Level: Mild
Image: 8881_right.jpeg, Predicted Severity Level: Mild
Image: 8888_left.jpeg, Predicted Severity Level: Severe
Image: 8882_left.jpeg, Predicted Severity Level: Mild
Image: 8888_right.jpeg, Predicted Severity Level: Severe
Image: 8883_left.jpeg, Predicted Severity Level: Mild
Image: 8889_right.jpeg, Predicted Severity Level: Mild
Image: 8881_left.jpeg, Predicted Severity Level: Mild
Image: 8900_left.jpeg, Predicted Severity Level: Severe
Image: 8912_left.jpeg, Predicted Severity Level: Mild
Image: 8900_right.jpeg, Predicted Severity Level: Mild
Image: 8896_right.jpeg, Predicted Severity Level: Mild
Image: 8906_right.jpeg, Predicted Severity Level: Severe
Image: 8

KeyboardInterrupt: ignored

In [ ]:
# """
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# import torchvision.transforms as transforms
# from torchvision.datasets import ImageFolder
# import torchvision.models as models

# # Set the path to your dataset
# data_dir = "/content/drive/MyDrive/resized_train_cropped"

# # Define the transformations applied to the input images
# transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# # Load the dataset
# dataset = ImageFolder(data_dir, transform=transform)

# # Define the DataLoader
# batch_size = 32
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Load the pre-trained DenseNet201 model
# model = models.densenet201(pretrained=True)

# # Replace the classifier to match the number of classes in your dataset
# num_classes = len(dataset.classes)
# in_features = model.classifier.in_features
# model.classifier = nn.Linear(in_features, num_classes)

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Check if a GPU is available and move the model to GPU if possible
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # Training loop
# num_epochs = 10

# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for images, labels in dataloader:
#         images = images.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     epoch_loss = running_loss / len(dataloader)
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# # Save the trained model
# torch.save(model.state_dict(), "trained_model2.pth")
# """

In [ ]:
# """
# import torch
# import torchvision.models as models
# import torchvision.transforms as transforms
# from PIL import Image
# # Create an instance of the DenseNet201 model
# model = models.densenet201()

# # Load the pretrained weights of your model
# pretrained_weights = torch.load("/content/trained_model1.pth")

# # Remove the 'module.' prefix from the state_dict keys (if present)
# if list(pretrained_weights.keys())[0].startswith('module.'):
#     pretrained_weights = {k[7:]: v for k, v in pretrained_weights.items()}

# # Remove the last layer of the current model classifier
# model.classifier = torch.nn.Identity()

# # Load the pretrained weights into the model (excluding the last layer)
# model.load_state_dict(pretrained_weights, strict=False)

# # Add a new classifier layer with the correct number of output classes
# num_classes = 5
# model.classifier = torch.nn.Linear(1920, num_classes)

# # Set the model to evaluation mode
# model.eval()
# # Preprocess the input image
# transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# # Load and preprocess the input image
# input_image = Image.open("/content/drive/MyDrive/test_img1.jpg")
# input_tensor = transform(input_image)
# input_batch = input_tensor.unsqueeze(0)

# # Check if a GPU is available and move the model and input to GPU if possible
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# input_batch = input_batch.to(device)

# # Perform the forward pass
# with torch.no_grad():
#     output = model(input_batch)

# # Get the predicted severity level
# _, predicted_index = torch.max(output, 1)

# severity_levels = ["No DR","Mild", "Moderate", "Severe", "Proliferative DR"]
# if predicted_index.item() < len(severity_levels):
#     predicted_severity = severity_levels[predicted_index.item()]
# else:
#     predicted_severity = "Unknown"

# print("Predicted Severity Level:", predicted_severity)
# """